# Analisi Dataset Dummy #

#### Libraries Import ####

In [34]:
import pandas as pd
from langdetect import detect
from IPython.display import display
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import re
from numpy import mean
from sklearn import metrics


import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('omw-1.4')

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split




from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Paolo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Paolo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Paolo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Paolo\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


#### DataSet Reading ####

In [35]:
df = pd.read_csv("C:/Users/Paolo/OneDrive - Universita' degli Studi di Roma Tor Vergata/Laurea Magistrale/Tesi/Codice/2.Classificazione/Dummy/Data/data_dummy.csv", sep='___', engine='python')


#### Check language of the news ####

In [4]:

total_news = 0
en_news = 0
it_news = 0
foreign_news = 0


for news in df['news']:
    if(detect(news) == 'en'):
        en_news += 1
        total_news += 1
    if(detect(news) == 'it'):
        it_news += 1
        total_news += 1   
    if(detect(news) != 'en' and detect(news) != 'it'):
        foreign_news += 1
        total_news += 1


print('Total News: ', total_news )
print('Italian News: ', it_news)
print('English News: ', en_news)
print('Foreign News: ', foreign_news)



Total News:  1034
Italian News:  74
English News:  936
Foreign News:  24


Keeping just the english ones (approx. 90%)

In [36]:

for news in df['news']:
    if(detect(news) == 'it' or detect(news) != 'en'):
        df = df.drop((df.index[df["news"]== news].tolist()))
 
total_news = 0
en_news = 0
it_news = 0
foreign_news = 0

for news in df['news']:
    if(detect(news) == 'en'):
        en_news += 1
        total_news += 1
    if(detect(news) == 'it'):
        it_news += 1
        total_news += 1   
    if(detect(news) != 'en' and detect(news) != 'it'):
        foreign_news += 1
        total_news += 1


print('Total News: ', total_news )
print('Italian News: ', it_news)
print('English News: ', en_news)
print('Foreign News: ', foreign_news)

df.reset_index(drop=True, inplace=True)

Total News:  936
Italian News:  0
English News:  936
Foreign News:  0


#### Checking the most common categories to make a 5-top selection ####

In [247]:
#df = df.dropna()
#df.reset_index(inplace=True, drop=True)
#Counter(" ".join(df["category"]).split()).most_common(10)
#Counter(" ".join(df["category.1"]).split()).most_common(10)
#Counter(" ".join(df["category.2"]).split()).most_common(10)
#Counter(" ".join(df["category.3"]).split()).most_common(10)
#Counter(" ".join(df["category.4"]).split()).most_common(10)




[('NotList', 281),
 ('OHCHR-Reject', 164),
 ('Health-Effects', 90),
 ('RIO_Scope', 40),
 ('Content-For-DPKR-NorthKorea', 38),
 ('ET_SportPT', 36),
 ('Firma_opslag', 34),
 ('GeneralEconomicKeywords', 30),
 ('WesternBalkans-Politics2', 28),
 ('ET_SPORTFR', 26)]

#### Define categories for all the news ####
Adding a new column where the definitive category will be written.

In [37]:
df = df.reindex(columns = df.columns.tolist() + ["cate_def"])


Iterate all over the rows to put the most relevant category (from the 5 selected) in the definitive slot. Let's just recap the categories are ordered by importance, from the most important (category) to the least important (category.4), so when the slot is fullfilled is never more touched.

In [38]:
count = 0

for cate0, cate1, cate2, cate3, cate4, catedef in zip(df['category'], df['category.1'], df['category.2'], df['category.3'], df['category.4'], df['cate_def']):
    if(cate0 == 'Coronavirus' or cate0 == 'GeneralEconomicKeywords' or cate0 == 'ET_SPORTFR' or cate0 == 'Health-Effects' or cate0 == 'Treatment'):
        df.loc[count, 'cate_def'] = cate0
    elif((cate1 == 'Coronavirus' or cate1 == 'GeneralEconomicKeywords' or cate1 == 'ET_SPORTFR' or cate1 == 'Health-Effects' or cate1 == 'Treatment') and pd.isna(catedef)):
        df.loc[count, 'cate_def'] = cate1
    elif((cate2 == 'Coronavirus' or cate2 == 'GeneralEconomicKeywords' or cate2 == 'ET_SPORTFR' or cate2 == 'Health-Effects' or cate2 == 'Treatment')and pd.isna(catedef)):
        df.loc[count, 'cate_def'] = cate2
    elif((cate3  == 'Coronavirus' or cate3 == 'GeneralEconomicKeywords' or cate3 == 'ET_SPORTFR' or cate3 == 'Health-Effects' or cate3 == 'Treatment')and pd.isna(catedef)):
        df.loc[count, 'cate_def'] = cate3
    elif((cate4 == 'Coronavirus' or cate4 == 'GeneralEconomicKeywords' or cate4 == 'ET_SPORTFR' or cate4 == 'Health-Effects' or cate4 == 'Treatment') and pd.isna(catedef)):
        df.loc[count, 'cate_def'] = cate4
    count = count + 1


Drop all the no more useful column

In [39]:

df.drop('category', inplace=True, axis=1)
df.drop('category.1', inplace=True, axis=1)
df.drop('category.2', inplace=True, axis=1)
df.drop('category.3', inplace=True, axis=1)
df.drop('category.4', inplace=True, axis=1)

df = df[df['cate_def'].notna()]
df.reset_index(drop=True, inplace=True)

### Export ###

In [40]:
df.rename(columns = {'cate_def':'category'}, inplace = True)

df['category_id'] = df['category'].factorize()[0]
df.head()


,title,source,news,category,category_id
0,3.6m indirect jobs created through $1.1bn externally funded projects –FG,https://www.sunnewsonline.com/3-6m-indirect-jobs-created-through-1-1bn-externally-funded-projects-fg/,"The Federal Government has said about 3.6 million indirect jobs have been created from $1.113 billion (about N461.9 billion) worth of externally funded projects being implemented by the present administration. Minister of Agriculture, Mohammed Abubakar, made the disclosure while appearing on a....",GeneralEconomicKeywords,0
1,"Merrifield says he's vaccinated, ready to play for Blue Jays",https://www.taiwannews.com.tw/en/news/4616894,MINNEAPOLIS (AP) — Whit Merrifield said Thursday he is vaccinated for COVID-19 and will be cleared to play for Toronto when the Blue Jays return to Canada. Merrifield was acquired in a trade with Kansas City on Tuesday for two minor leaguers. The two-time All-Star was in the starting lineup in....,Treatment,1
2,"Steelers, WR Diontae Johnson agree to 3-year deal",https://www.taiwannews.com.tw/en/news/4616770,"LATROBE, Pa. (AP) — Diontae Johnson watched the wide receiver market explode during the offseason. The dollar amounts were so staggering they were impossible for Johnson to ignore as he entered contract negotiations with the Pittsburgh Steelers. “You see the numbers,” Johnson said. “But I wasn’t trying to look at everybody’s pockets.",ET_SPORTFR,2
3,"N.Korea marks end of first COVID wave, but risks persist By Hyonhee Shin and Soo-hyang Choi SEOUL (Reuters) - North Korea on Friday said all of its fever patients have recovered, marking the end of its first... 55m ago",https://kelo.com/2022/08/04/n-korea-marks-end-of-first-covid-wave-but-risks-persist/,"By Hyonhee Shin and Soo-hyang Choi. SEOUL (Reuters) – North Korea on Friday said all of its fever patients have recovered, marking the end of its first wave of the coronavirus pandemic, but challenges linger with economic hardships deepening and an unvaccinated population exposed to future resurgences, analysts said.",Coronavirus,3
4,Novak Djokovic social media row erupts Australian tennis…,https://samacharcentral.com/novak-djokovic-social-media-row-erupts-australian-tennis-star-feels-heat-for-backing-serb-tennis-sport/,"Australian tennis star John Millman has questioned on Twitter why Novak Djokovic cannot participate in the US Open later this month after he himself caught Covid in the country. However, he was forced to bat away the backlash with several follow-up tweets.",ET_SPORTFR,2


In [41]:
tsv = df.to_csv(sep='\1', index=False) # use '\1' if your data contains tabs
psv = tsv.replace('\1', '___')
with open("C:/Users/Paolo/OneDrive - Universita' degli Studi di Roma Tor Vergata/Laurea Magistrale/Tesi/Codice/2.Classificazione/Dummy/Data/data_dummy_cleaned_for_nn.csv", 'w', encoding='utf-8', newline='\n') as outfile:
    outfile.write(psv)